In [571]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [778]:
import os
import os.path
import pandas as pd
import numpy as np
import sys
import re
import sys
import json

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

pd.set_option('display.max_columns', 50)

In [779]:
from cig_script.config.config_env import Config

from cig_script.mail_copy_input_format import _get_mul_car_one_bl
from cig_script.mail_copy_input_format import _get_one_car_one_bl
from cig_script.mail_copy_input_format import _autowidth_excel
from datetime import datetime





In [780]:
mail_copy_in_path = Config().mail_copy_in_path
one_car_one_bl_df = _get_one_car_one_bl(mail_copy_in_path)
mul_car_one_bl_df = _get_mul_car_one_bl(mail_copy_in_path)


In [781]:
total_option = input('1 : Y(Yes) or  0: N(No) : ')

1 : Y(Yes) or  0: N(No) : 1


In [782]:
total_option = total_option.upper()
total_option

'1'

In [783]:
def _total_excel_update(one_car_df,mul_car_df,mail_copy_in_path):
    now = datetime.now()
    gen_time = now.strftime('%Y-%m-%d %H:%M')
    
    tot_excel_from_mail = Config().tot_excel_from_mail
    tot_one_car_sheet = Config().total_one_car_sheet_name
    tot_mul_car_sheet = Config().total_mul_car_sheet_name
    
    one_car_df['gen_time'] = gen_time
    mul_car_df['gen_time'] = gen_time
    

    if os.path.isfile(tot_excel_from_mail):
        tot_one_car_df = pd.read_excel(tot_excel_from_mail,sheet_name=tot_one_car_sheet)
        tot_mul_car_df = pd.read_excel(tot_excel_from_mail,sheet_name=tot_mul_car_sheet)
        
        _is_include_one_car,exist_dup_one_car,sub_one_car_df = _check_include_df(one_car_df,tot_one_car_df,gen_time)
        _is_include_mul_car,exist_dup_mul_car,sub_mul_car_df = _check_include_df(mul_car_df,tot_mul_car_df,gen_time)
        
        if _is_include_one_car and _is_include_mul_car:
            pass
            # both already included.
        elif _is_include_mul_car:
            # multi car already included, one car should be updated
            if exist_dup_one_car:
                tot_one_car_df = pd.concat([tot_one_car_df,sub_one_car_df]).set_index('YR').reset_index()
            else:
                tot_one_car_df = pd.concat([tot_one_car_df,one_car_df]).set_index('YR').reset_index()
            
            tot_one_car_df = tot_one_car_df.astype(str)
            tot_mul_car_df = tot_mul_car_df.astype(str)
            
            with pd.ExcelWriter(tot_excel_from_mail,engine='xlsxwriter') as writer:
                tot_one_car_df.to_excel(writer,tot_one_car_sheet,index=False)
                _autowidth_excel(writer,tot_one_car_sheet,tot_one_car_df)
                tot_mul_car_df.to_excel(writer,tot_mul_car_sheet,index=False)
                _autowidth_excel(writer,tot_mul_car_sheet,tot_mul_car_df)
                
        elif _is_include_one_car:
            # one car already included, multi car should be updated
            if exist_dup_mul_car:
                tot_mul_car_df = pd.concat([tot_mul_car_df,sub_mul_car_df]).set_index('YR').reset_index()
            else:
                tot_mul_car_df = pd.concat([tot_mul_car_df,mul_car_df]).set_index('YR').reset_index()
            
            tot_one_car_df = tot_one_car_df.astype(str)
            tot_mul_car_df = tot_mul_car_df.astype(str)
            
            with pd.ExcelWriter(tot_excel_from_mail,engine='xlsxwriter') as writer:
                tot_one_car_df.to_excel(writer,tot_one_car_sheet,index=False)
                _autowidth_excel(writer,tot_one_car_sheet,tot_one_car_df)
                tot_mul_car_df.to_excel(writer,tot_mul_car_sheet,index=False)
                _autowidth_excel(writer,tot_mul_car_sheet,tot_mul_car_df)
            
        else:
            if exist_dup_one_car and exist_dup_mul_car:
                tot_one_car_df = pd.concat([tot_one_car_df,sub_one_car_df]).set_index('YR').reset_index()
                tot_mul_car_df = pd.concat([tot_mul_car_df,sub_mul_car_df]).set_index('YR').reset_index()
            elif exist_dup_one_car:
                tot_one_car_df = pd.concat([tot_one_car_df,sub_one_car_df]).set_index('YR').reset_index()
                tot_mul_car_df = pd.concat([tot_mul_car_df,mul_car_df]).set_index('YR').reset_index()
            elif exist_dup_mul_car:
                tot_one_car_df = pd.concat([tot_one_car_df,one_car_df]).set_index('YR').reset_index()
                tot_mul_car_df = pd.concat([tot_mul_car_df,sub_mul_car_df]).set_index('YR').reset_index()
            else:
                tot_one_car_df = pd.concat([tot_one_car_df,one_car_df]).set_index('YR').reset_index()
                tot_mul_car_df = pd.concat([tot_mul_car_df,mul_car_df]).set_index('YR').reset_index()
            
            tot_one_car_df = tot_one_car_df.astype(str)
            tot_mul_car_df = tot_mul_car_df.astype(str)
            
            with pd.ExcelWriter(tot_excel_from_mail,engine='xlsxwriter') as writer:
                tot_one_car_df.to_excel(writer,tot_one_car_sheet,index=False)
                _autowidth_excel(writer,tot_one_car_sheet,tot_one_car_df)
                tot_mul_car_df.to_excel(writer,tot_mul_car_sheet,index=False)
                _autowidth_excel(writer,tot_mul_car_sheet,tot_mul_car_df)
        
    else:
        with pd.ExcelWriter(tot_excel_from_mail,engine='xlsxwriter') as writer:
            
            one_car_df.to_excel(writer,tot_one_car_sheet,index=False)
            _autowidth_excel(writer,tot_one_car_sheet,one_car_df)

            mul_car_df.to_excel(writer,tot_mul_car_sheet,index=False)
            _autowidth_excel(writer,tot_mul_car_sheet,mul_car_df)
    

In [784]:
def _check_include_df(cur_df,tot_df,gen_time):
    exist_dup = False
    
    cur_df = cur_df.set_index('YR').reset_index()
    cur_df = cur_df.copy().drop(columns=['gen_time'])
    tot_df = tot_df.copy().drop(columns=['gen_time'])
    
    cols = list(cur_df.columns)
2
    cur_df = cur_df.astype(str)
    tot_df = tot_df.astype(str)

    common_df = pd.merge(cur_df,tot_df, on=cols)
    is_include = cur_df.equals(common_df)  
    
    cur_idx_set = set(cur_df.index)
    common_idx_set = set(common_df.index)
    sub_idx_list = list(cur_idx_set - common_idx_set)
    
    sub_df = cur_df.copy().iloc[sub_idx_list,:]
    sub_df['gen_time'] = gen_time
    
    if is_include:
        pass
    else:
        if len(cur_df) != len(common_df):
            exist_dup = True
        else:
            exist_dup = False
    
    return is_include,exist_dup,sub_df

In [785]:
if total_option in ['1', 'Y','YES']:
    _total_excel_update(one_car_one_bl_df,mul_car_one_bl_df,mail_copy_in_path)
else:
    print('nokay')